# How to route between sub-chains

# 1. Using a Custom Function (Recommended)
# Steps:
 * Classification Step: Use a chain to classify input into categories (e.g., "LangChain", "Anthropic", or "Other").
 * Sub-Chains: Define separate sub-chains for handling each category.
 * Custom Routing Function:
      * Define a function that takes input, evaluates conditions, and returns the appropriate sub-chain.
 * Combine Chains: Use RunnableLambda to integrate the routing logic.

In [2]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnableLambda

# Define a prompt template
prompt = PromptTemplate(input_variables=["question"], template="Answer the following question: {question}")

# Create an LLM chain
llm = ChatOpenAI(temperature=0)
chain = LLMChain(llm=llm, prompt=prompt)

# Define a routing function
def route(inputs):
    question = inputs["question"]
    if "Anthropic" in question:
        return {"answer": "This is an answer related to Anthropic"}
    return chain.invoke(inputs)

# Combine with a RunnableLambda
full_chain = {
    "topic": chain,  # Replace this if "topic" has another purpose
    "question": lambda x: x["question"]
} | RunnableLambda(route)

# Invoke the chain with input
response = full_chain.invoke({"question": "how do I use Anthropic?"})
print(response)


C:\Users\Admin\AppData\Local\Temp\ipykernel_2912\3797212930.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_2912\3797212930.py:11: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


{'answer': 'This is an answer related to Anthropic'}


# 2. Using RunnableBranch (Legacy)
# Steps:
* Define Branch Conditions: Specify a list of (condition, runnable) pairs.
* Default Chain: Include a fallback runnable if no conditions match.
* Combine with Classifier: Integrate with the classification chain.

In [4]:
from langchain_core.runnables import RunnableBranch
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

# Define LLM
llm = ChatOpenAI(temperature=0)

# Define chains for specific topics
anthropic_prompt = PromptTemplate(
    input_variables=["question"], template="This is a question about Anthropic: {question}"
)
anthropic_chain = LLMChain(llm=llm, prompt=anthropic_prompt)

langchain_prompt = PromptTemplate(
    input_variables=["question"], template="This is a question about LangChain: {question}"
)
langchain_chain = LLMChain(llm=llm, prompt=langchain_prompt)

# Define a general fallback chain
general_prompt = PromptTemplate(
    input_variables=["question"], template="Answer this question: {question}"
)
general_chain = LLMChain(llm=llm, prompt=general_prompt)

# Define branch logic
branch = RunnableBranch(
    (lambda x: "anthropic" in x["topic"].lower(), anthropic_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain,  # Default chain
)

# Combine with classifier
full_chain = {"topic": lambda x: x["topic"], "question": lambda x: x["question"]} | branch

# Test the chain
response = full_chain.invoke({"topic": "Anthropic", "question": "What is Anthropic?"})
print(response)


{'topic': 'Anthropic', 'question': 'What is Anthropic?', 'text': 'Anthropic refers to the principle that the universe and its physical laws are finely tuned to allow for the existence of intelligent life. This concept suggests that the fundamental constants and parameters of the universe are precisely set in a way that allows for the emergence of life as we know it. The anthropic principle has been a topic of debate among scientists and philosophers, with some arguing that it points to a higher purpose or design in the universe, while others see it as a natural consequence of the laws of physics.'}


Explanation:

anthropic_chain: Handles inputs related to "Anthropic" using a specific prompt.

langchain_chain: Handles inputs related to "LangChain."

general_chain: Acts as a fallback for other topics.

RunnableBranch: Directs inputs to the correct chain based on the topic.

Combining Chains: The full_chain ensures the routing logic works seamlessly.